In [ ]:
import os
import pandas as pd
import matplotlib.ticker as mticker
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibratedClassifierCV

### Load in relevant data

In [ ]:
training_data = pd.read_csv('final\\2b. Compare Cox, ML\\CoxMLTraining_totalModel.csv')
testing_data = pd.read_csv('final\\2b. Compare Cox, ML\\CoxMLTesting_totalModel.csv')

In [ ]:
# drop ethnicity and townsend (and patient ID) as predictors for aim 2
training_data = training_data[["baseline_age", "followup_age", "followup_time", "gender_encoded", "height (cm)", "weight (kg)", "alcohol_status", "smoking_status", "diabetes_status", "antiplatelets_use", "crp value (mg/L)", "total_outcomes"]]
testing_data = testing_data[["baseline_age", "followup_age", "followup_time", "gender_encoded", "height (cm)", "weight (kg)", "alcohol_status", "smoking_status", "diabetes_status", "antiplatelets_use", "crp value (mg/L)", "total_outcomes"]]

### Split data into X (predictors) and Y (target)

In [ ]:
X_train = training_data.iloc[:,1:-1].values
y_train = training_data.iloc[:,-1].values

X_test = testing_data.iloc[:,1:-1].values
y_test = testing_data.iloc[:,-1].values

### Make pipeline with StandardScaler and ML model of choice

In [ ]:
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

### Set parameters for gridsearchCV

In [ ]:
param_grid = {
    'randomforestclassifier__random_state': [1024],
    'randomforestclassifier__n_jobs': [1],
    'randomforestclassifier__n_estimators': [100],
    'randomforestclassifier__criterion': ['gini'],
    'randomforestclassifier__oob_score': [True],
    'randomforestclassifier__max_features': [0.1, 0.2, 0.4, 0.8],
    'randomforestclassifier__max_depth': [4, 8, 12],
    'randomforestclassifier__class_weight': ['balanced']
}

### Perform gridsearchCV and get best model

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv = 10)
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

### Evaluate model on test set and get scores

#### AUC

In [ ]:
# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred) 
classification_rep = classification_report(y_test, y_pred)
print("Test set AUC score:", auc_score)
print("Classification report:\n", classification_rep)

In [ ]:
# AUC confidence intervals
from sklearn.metrics import roc_auc_score
from math import sqrt

def roc_auc_ci(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, upper)

roc_auc_ci(y_test, y_pred, positive=1)

#### calibration plots

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from sklearn.calibration import calibration_curve

# Get the predicted probabilities from the best_model
y_pred_probs = best_model.predict_proba(X_test)[:, 1]

# Calculate the true probabilities using calibration_curve
true_probs, pred_probs = calibration_curve(y_test, y_pred_probs, n_bins=10)

# Create a new figure for the calibration plot
fig_calibration = plt.figure(figsize=(10, 10))
ax_calibration_curve = fig_calibration.add_subplot(111)

# Plot the calibration curve
ax_calibration_curve.plot(pred_probs, true_probs, marker='o')

# Plot the diagonal line for perfect calibration
ax_calibration_curve.plot([0, 1], [0, 1], linestyle='--', color='gray')

# Customize the plot if needed
ax_calibration_curve.grid(False)
ax_calibration_curve.set_title("Calibration: Random Forest")
ax_calibration_curve.set_xlabel("Predicted 10-year risk")
ax_calibration_curve.set_ylabel("Observed 10-year risk")

# Define the tick formatter function
def percent_formatter(x, pos):
    return "{:.0%}".format(x)

# Apply the tick formatter to the x and y axes
ax_calibration_curve.xaxis.set_major_formatter(mticker.FuncFormatter(percent_formatter))
ax_calibration_curve.yaxis.set_major_formatter(mticker.FuncFormatter(percent_formatter))

text = "C-statistic: 0.874 (95%CI 0.869-0.879)" 
ax_calibration_curve.text(0.05, 0.95, text, transform=ax_calibration_curve.transAxes,
                          fontsize=12, verticalalignment='top', horizontalalignment='left')

# Show the plot
plt.show()

#### histogram of predicted risks

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Get the predicted risks from the best_model
y_pred_probs = best_model.predict_proba(X_test)[:, 1]

# Create a new figure for the histogram
fig_histogram = plt.figure(figsize=(8, 6))
ax_histogram = fig_histogram.add_subplot(111)

# Plot the histogram of predicted risks with grey bins
n, bins, patches = ax_histogram.hist(y_pred_probs, bins=10, edgecolor='black', color='grey')

# Set the y-axis tick formatter as percentage
ax_histogram.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=len(y_pred_probs)))

ax_histogram.set_title("Distribution of Predicted Risk: Random Forest")
ax_histogram.set_xlabel("Predicted Risk")
ax_histogram.set_ylabel("Percent")

# Show the histogram plot
plt.show()

#### get mean and range of predicted risks

In [ ]:
np.mean(y_pred_probs) 
np.min(y_pred_probs) 
np.max(y_pred_probs) 